In [ ]:
import cv2
import numpy as np
import sys
import time

# --- find_camera_index 함수는 동일하므로 생략 ---
def find_camera_index():
    """
    시스템에 연결된 유효한 카메라 인덱스를 찾아 리스트로 반환합니다.
    (이전 코드와 동일)
    """
    print("--- 📸 사용 가능한 카메라 인덱스 테스트 중 ---")
    available_indices = []
    
    for i in range(3): 
        # CAP_DSHOW 백엔드는 Windows 환경에서 카메라 열기 안정성을 높일 수 있습니다.
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW) 
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                print(f"✅ 인덱스 {i}: 성공적으로 카메라를 열고 프레임 읽기 성공!")
                available_indices.append(i)
                cap.release()
            else:
                cap.release()
                print(f"⚠️ 인덱스 {i}: 카메라를 열었으나 프레임 읽기 실패.")
        else:
            print(f"❌ 인덱스 {i}: 카메라 열기 실패. 이 인덱스에는 장치가 없습니다.")
            pass # 모든 인덱스를 시도하도록 유지
                 
    if not available_indices:
        print("\n**오류:** 시스템에서 유효한 카메라 장치를 찾지 못했습니다. 드라이버/연결 상태를 확인하세요.")
    else:
        print(f"\n✅ 유효한 카메라 인덱스 목록: {available_indices}")
        
    return available_indices
# ---------------------------------------------------


def live_bbox_center_detector(camera_index):
    """
    실시간 카메라 스트림에서 흰색 및 초록색 물체를 감지하고 중앙 픽셀 좌표를 계산하여 출력합니다.
    """
    cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW) 
    
    if not cap.isOpened():
        print(f"FATAL ERROR: 카메라 인덱스 {camera_index}를 열 수 없습니다. 프로그램을 종료합니다.")
        sys.exit(1)

    # ----------------------------------------------------
    # **수정된 상자 감지 로직 (흰색 + 초록색 감지)**
    # ----------------------------------------------------
    
    # 1. 초록색 (Green) 범위
    # H: 35~85 (녹색 계열), S: 50~255 (채도 높음), V: 50~255 (명도 높음)
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([85, 255, 255]) 
    
    # 2. 흰색 (White) 범위
    # 흰색은 H(색상)보다 S(채도)와 V(명도)에 의존합니다.
    # S가 매우 낮고 V가 매우 높은 영역을 설정합니다.
    lower_white = np.array([0, 0, 250])   # S: 0부터 시작 (무채색), V: 200 이상 (매우 밝음)
    upper_white = np.array([180, 50, 255]) # S: 50 미만 (채도 낮음)
    
    print(f"--- 🚀 실시간 좌표 감지 시작 (인덱스 {camera_index}, Q 키를 눌러 종료) ---")
    print(f"Width: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))} Height: {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 받을 수 없습니다. 루프를 종료합니다.", end='\r')
            time.sleep(0.1)
            continue

        # BGR을 HSV로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # 초록색 마스크 생성
        mask_green = cv2.inRange(hsv, lower_green, upper_green)
        
        # 흰색 마스크 생성
        mask_white = cv2.inRange(hsv, lower_white, upper_white)
        
        # 두 마스크를 합쳐 최종 마스크 생성 (OR 연산)
        final_mask = cv2.bitwise_or(mask_green, mask_white)
        
        # Erosion/Dilation 같은 모폴로지 연산을 추가하면 노이즈 제거에 도움됨 (선택 사항)
        
        # 2. 상자 감지 및 바운딩 박스 추출
        # 합쳐진 마스크에서 컨투어 찾기
        contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        target_center = None
        
        if contours:
            # 면적이 가장 큰 컨투어 선택 (상자가 하나만 있다고 가정)
            largest_contour = max(contours, key=cv2.contourArea)
            
            # 노이즈를 걸러내기 위한 최소 면적 설정
            if cv2.contourArea(largest_contour) > 1000:
                
                # 바운딩 박스 (u_min, v_min, w, h)
                u_min, v_min, w, h = cv2.boundingRect(largest_contour)
                u_max = u_min + w
                v_max = v_min + h
                
                # 3. 픽업 중앙 픽셀 좌표 (u_c, v_c) 계산
                u_c = int((u_min + u_max) / 2)
                v_c = int((v_min + v_max) / 2)
                target_center = (u_c, v_c)

                # 4. 시각화
                cv2.rectangle(frame, (u_min, v_min), (u_max, v_max), (0, 0, 255), 2)
                cv2.circle(frame, target_center, 5, (0, 0, 255), -1)
                text = f"P: ({u_c}, {v_c})"
                cv2.putText(frame, text, (u_min, v_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                # 5. 로봇 팀에 전달할 핵심 데이터 출력
                print(f"🎯 좌표 감지: u={u_c}, v={v_c} (Index: {camera_index})", end='\r')
            else:
                print("상자 감지 안 됨 (면적 부족)", end='\r')
        else:
            print("상자 감지 안 됨 (컨투어 없음)", end='\r')

        cv2.imshow('Live Box Detection & Pick-up Point', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    
    # 1. 유효한 카메라 인덱스 찾기
    available_indices = find_camera_index()
    
    # 2. 사용할 카메라 인덱스 설정
    if not available_indices:
        print("\n유효한 카메라를 찾지 못했습니다. 프로그램을 종료합니다.")
        sys.exit(0)
    
    # 사용자 정의: 리스트의 세 번째 인덱스(Index 2)를 선택
    # 만약 available_indices 리스트에 3개 미만의 요소가 있다면 IndexError가 발생할 수 있습니다.
    # 안전하게 사용 가능한 인덱스를 선택하도록 수정합니다.
    if len(available_indices) > 2:
        TARGET_CAMERA_INDEX = available_indices[2] 
    else:
        # 2번 인덱스가 없다면, 첫 번째 인덱스(0)를 사용
        TARGET_CAMERA_INDEX = available_indices[0] 
        print(f"\n⚠️ 주의: 2번 인덱스가 없어, {TARGET_CAMERA_INDEX}번 인덱스를 대신 사용합니다.")
    
    # **만약 myCobot 카메라 인덱스를 직접 알고 있다면 여기에 수동으로 지정하세요.**
    # 예시: TARGET_CAMERA_INDEX = 1 
    
    print(f"\n>> 사용할 카메라 인덱스: {TARGET_CAMERA_INDEX}")
    
    # 3. 실시간 감지 시작
    live_bbox_center_detector(TARGET_CAMERA_INDEX)

In [ ]:
import cv2
import numpy as np
import sys
import time

# --- find_camera_index 함수는 동일하므로 생략 ---
def find_camera_index():
    """
    시스템에 연결된 유효한 카메라 인덱스를 찾아 리스트로 반환합니다.
    """
    print("--- 📸 사용 가능한 카메라 인덱스 테스트 중 ---")
    available_indices = []
    
    for i in range(3): 
        # CAP_DSHOW 백엔드는 Windows 환경에서 카메라 열기 안정성을 높일 수 있습니다.
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW) 
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                print(f"✅ 인덱스 {i}: 성공적으로 카메라를 열고 프레임 읽기 성공!")
                available_indices.append(i)
                cap.release()
            else:
                cap.release()
                print(f"⚠️ 인덱스 {i}: 카메라를 열었으나 프레임 읽기 실패.")
        else:
            print(f"❌ 인덱스 {i}: 카메라 열기 실패. 이 인덱스에는 장치가 없습니다.")
            pass
                 
    if not available_indices:
        print("\n**오류:** 시스템에서 유효한 카메라 장치를 찾지 못했습니다. 드라이버/연결 상태를 확인하세요.")
    else:
        print(f"\n✅ 유효한 카메라 인덱스 목록: {available_indices}")
        
    return available_indices
# ---------------------------------------------------


def live_bbox_center_detector(camera_index):
    """
    실시간 카메라 스트림에서 흰색 및 초록색 물체를 감지하고 
    중앙 픽셀 좌표 (u, v) 및 Roll 회전 각도를 계산하여 출력합니다.
    """
    cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW) 
    
    if not cap.isOpened():
        print(f"FATAL ERROR: 카메라 인덱스 {camera_index}를 열 수 없습니다. 프로그램을 종료합니다.")
        sys.exit(1)

    # ----------------------------------------------------
    # **색상 감지 범위 (흰색 + 초록색 감지)**
    # ----------------------------------------------------
    
    # 1. 초록색 (Green) 범위
    # H: 35~85 (녹색 계열), S: 50~255 (채도 높음), V: 50~255 (명도 높음)
    lower_green = np.array([35, 50, 50])
    upper_green = np.array([85, 255, 255]) 
    
    # 2. 흰색 (White) 범위
    # 흰색은 낮은 채도(S)와 높은 명도(V)에 의존
    lower_white = np.array([0, 0, 250])   # V: 200 이상 (매우 밝음)
    upper_white = np.array([180, 50, 255]) # S: 50 미만 (채도 낮음)
    
    # 시각화 색상 설정 (BGR: 빨간색)
    COLOR_RED = (0, 0, 255)
    
    print(f"--- 🚀 실시간 좌표 및 Roll 감지 시작 (인덱스 {camera_index}, Q 키를 눌러 종료) ---")
    
    # 현재 스트리밍 해상도 출력
    actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"🖼️ 현재 스트리밍 해상도: {actual_width} (가로) x {actual_height} (세로)")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 받을 수 없습니다. 루프를 종료합니다.", end='\r')
            time.sleep(0.1)
            continue

        # BGR을 HSV로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # 마스크 생성 및 합치기
        mask_green = cv2.inRange(hsv, lower_green, upper_green)
        mask_white = cv2.inRange(hsv, lower_white, upper_white)
        final_mask = cv2.bitwise_or(mask_green, mask_white)
        
        # 모폴로지 연산 (작은 노이즈 제거) - 선택 사항
        # kernel = np.ones((5, 5), np.uint8)
        # final_mask = cv2.erode(final_mask, kernel, iterations=1)
        # final_mask = cv2.dilate(final_mask, kernel, iterations=1)
        
        # 2. 상자 감지 및 바운딩 박스 추출
        contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        u_c, v_c, roll_angle = None, None, None
        
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            
            # 노이즈를 걸러내기 위한 최소 면적 설정
            if cv2.contourArea(largest_contour) > 1000:
                
                # 3. Roll 값 계산: 최소 회전 바운딩 사각형 사용
                rect = cv2.minAreaRect(largest_contour)
                
                # rect = ((u_c, v_c), (width, height), angle)
                (u_c_float, v_c_float), (w, h), angle = rect
                
                u_c = int(u_c_float)
                v_c = int(v_c_float)
                
                # OpenCV의 minAreaRect 각도 보정 로직 (로봇 팔에 전달하기 위해 0~180 또는 -90~90 범위로 정규화)
                # Roll 각도는 수평축(x축)과 긴 변 사이의 각도로 정의하는 것이 일반적입니다.
                if w < h:
                    roll_angle = 90 + angle
                else:
                    roll_angle = angle
                
                # Roll 각도 값은 보통 -90.00 ~ 0.00 또는 0.00 ~ 90.00 사이의 값으로 표현됩니다.
                
                # 4. 시각화
                # 회전 바운딩 박스 그리기
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(frame, [box], 0, COLOR_RED, 2)
                
                # 중심점 및 텍스트 표시
                cv2.circle(frame, (u_c, v_c), 5, COLOR_RED, -1)
                text = f"P: ({u_c}, {v_c}) | Roll: {roll_angle:.2f}°"
                
                # 텍스트 위치를 중심으로 부터 약간 위로 조정
                text_position = (u_c - 100, v_c - 30) 
                cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLOR_RED, 2)

                # 5. 로봇 팀에 전달할 핵심 데이터 출력
                print(f"🎯 감지: u={u_c}, v={v_c}, Roll={roll_angle:.2f}° (Index: {camera_index})", end='\r')
            else:
                print("상자 감지 안 됨 (면적 부족)", end='\r')
        else:
            print("상자 감지 안 됨 (컨투어 없음)", end='\r')

        cv2.imshow('Live Box Detection, Pick-up Point & Roll Angle', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    
    # 1. 유효한 카메라 인덱스 찾기
    available_indices = find_camera_index()
    
    # 2. 사용할 카메라 인덱스 설정
    if not available_indices:
        print("\n유효한 카메라를 찾지 못했습니다. 프로그램을 종료합니다.")
        sys.exit(0)
    
    # 안전하게 사용 가능한 인덱스를 선택하도록 수정
    if len(available_indices) > 2:
        TARGET_CAMERA_INDEX = available_indices[2] 
    else:
        # 2번 인덱스가 없다면, 첫 번째 인덱스(0)를 사용
        TARGET_CAMERA_INDEX = available_indices[0] 
        print(f"\n⚠️ 주의: 2번 인덱스가 없어, {TARGET_CAMERA_INDEX}번 인덱스를 대신 사용합니다.")
    
    print(f"\n>> 사용할 카메라 인덱스: {TARGET_CAMERA_INDEX}")
    
    # 3. 실시간 감지 시작
    live_bbox_center_detector(TARGET_CAMERA_INDEX)
